In [ ]:
from copy import deepcopy, copy
import os
import pathlib
from random import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, ConcatDataset, WeightedRandomSampler
from livelossplot import PlotLosses
from sklearn.model_selection import train_test_split

from chord_recognition.augmentations import SemitoneShift, DetuningShift
from chord_recognition.cache import HDF5Cache
from chord_recognition.cnn import model
from chord_recognition.dataset import ChromaDataset, prepare_datasource, split_datasource, get_weighted_random_sampler

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 5)

%load_ext autoreload
%autoreload 2

In [2]:
#datasource = prepare_datasource(('queen', 'beatles', 'robbie_williams', 'zweieck'))
datasource = prepare_datasource(('beatles', ))
dataset = ChromaDataset(
    datasource, window_size=8192, hop_length=4096,
    cache=HDF5Cache('chroma_cache.hdf5'))

In [3]:
# Split dataset into train/val keeping equal proportions of the each class
# Now train and val datasets have equal class probability distributions.
X = [sample for sample, _ in dataset]
y = [target for _, target in dataset]
indices = np.arange(len(X))
X_train, _, y_train, _, idx_train, idx_val = train_test_split(
    X, y, indices, test_size=0.2, stratify=y)

# Balance the classes in each batch which hopefully helps the training.
class_idxs = [x[1] for x in dataset]
_, class_counts = np.unique(class_idxs, return_counts=True)
sampler = get_weighted_random_sampler(zip(X_train, y_train), class_counts)
del X, y, X_train, y_train

In [4]:
len(idx_train), len(idx_val)

(71839, 17960)

In [5]:
# Split dataset into train/val datasets
# Make sure that train and val datasets have equal class probability distributions.
loader_train = DataLoader(
    dataset=dataset[idx_train],
    batch_size=128,
    sampler=sampler,
    pin_memory=False,
    num_workers=0)
loader_val = DataLoader(
    dataset=dataset[idx_val],
    num_workers=0,
    batch_size=25)

dataloaders = {
    "train": loader_train,
    "val": loader_val
}

In [6]:
def save_checkpoint(state_dict, is_best, filename='best_model.pt'):
    if is_best:
        torch.save(state_dict, f'chord_recognition/models/{filename}')

In [7]:
def train_model(model, optimizer, dataloaders, device, epochs=1):
    liveloss = PlotLosses()
    model = model.to(device=device)
    
    best_acc = 0
    for e in range(epochs):
        logs = {}
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # put model to training mode
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
        
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device=device, dtype=torch.float32)
                labels = labels.to(device=device, dtype=torch.long)
                if phase == 'train':
                    # Zero out all of the gradients for the variables which the optimizer
                    # will update.
                    optimizer.zero_grad()
            
                scores = model(inputs)
                scores = scores.squeeze(3).squeeze(2)                
                loss = F.cross_entropy(scores, labels)
                
                _, preds = torch.max(scores, 1)
                running_corrects += torch.sum(preds == labels)

        
                if phase == 'train':        
                    # This is the backwards pass: compute the gradient of the loss with
                    # respect to each  parameter of the model.
                    loss.backward()
        
                    # Actually update the parameters of the model using the gradients
                    # computed by the backwards pass.
                    optimizer.step()
            
                running_loss += loss.detach() * inputs.size(0)
        
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)
            prefix = ''
            if phase == 'val':
                prefix = 'val_'
                is_best = epoch_acc > best_acc
                best_acc = max(epoch_acc, best_acc)
                save_checkpoint(model.state_dict(), is_best)
            
            logs[prefix + ' log loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        
        liveloss.update(logs)
        liveloss.send()

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))

train_model(model, optimizer, dataloaders, device, 100)

torch.Size([128, 1, 119, 15])
tensor([24, 14,  4, 13,  6,  1, 22, 11, 19, 18,  0, 16,  1,  6, 21, 20,  4,  6,
         7,  4, 17,  5,  3, 19,  4,  2,  1, 20, 13,  2,  9, 14,  8, 10,  0,  3,
         1, 20, 23,  0, 18, 20, 10, 20, 21,  2, 12,  8,  2, 19,  2,  4, 21, 13,
         4,  6, 16,  7, 11,  0,  8, 16, 12,  2, 18, 11, 20, 19, 18, 21, 22, 21,
        12,  2, 11, 13, 10, 15, 15,  2,  9, 19, 20,  9, 10, 18, 14, 20,  1, 19,
        14, 20, 22, 15, 12,  6,  7, 13, 19,  9, 10, 18,  6, 21, 10,  5, 19,  4,
        10,  9,  7, 19, 14, 21, 23,  5, 23, 13, 17,  6, 15, 20,  1, 13, 19, 16,
        21, 24])


AttributeError: 'int' object has no attribute 'float'

In [ ]:
# Eliminate overfitting:
# - make sure that training and validation datasets have equal class probability distributions.
# - make sure that batch accepts accepts all classes
# - Try CNN -> CTC loss https://www.cs.toronto.edu/~graves/icml_2006.pdf
# - try CNN for extraction a sequence of features and vanilla RNN to propagate information through this sequence.